In [1]:
import pyodbc
import pandas as pd
pd.set_option('display.max_columns', None)                          # Unfolding hidden features if the cardinality is high      
pd.set_option('display.max_colwidth', None)                         # Unfolding the max feature width for better clearity      
pd.set_option('display.max_rows', None)                            # Unfolding hidden data points if the cardinality is high
pd.set_option('display.float_format', lambda x: '%.2f' % x)         # To suppress scientific notation over exponential values
import numpy as np
import pathlib
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import textwrap
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore") 

In [2]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)']

In [3]:
connection_string = "DRIVER={SQL Server Native Client 11.0};SERVER=gsc-scpat-sql-001-d.database.windows.net;DATABASE=SC-PAT-DB;UID=SCPAT;PWD=Ecolab@1234"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

conn = create_engine(connection_url)

In [4]:
conn

Engine(mssql+pyodbc://?odbc_connect=DRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3Dgsc-scpat-sql-001-d.database.windows.net%3BDATABASE%3DSC-PAT-DB%3BUID%3DSCPAT%3BPWD%3DEcolab%401234)

In [5]:
data_query = textwrap.dedent( """ SELECT top(1000000) [MD_Error]
,[Date]
,SUM(cast([ABS_STAT] as float)) AS [ABS_STAT],SUM(cast([ABS_CONS] as float)) AS [ABS_CONS]
,[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc]
,sum(cast([Consensus] as float)) as [Consensus],sum(cast([Stat]as float)) as [Stat],sum(cast([Actual]as float)) as [Actual]
,[Origin]
FROM [FABIAS_Dashboard].[FALossBaseCal]
where [Date] > Dateadd(Month, -13, getdate()) and [Date] < Dateadd(Month, -1,getdate())
-- WHERE [Item] = '74700PLUS.61R' and [Lag] = 'Lag 2' and [Period] = '202101'
GROUP BY
[MD_Error],[Date],[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc],[Origin]
  """)

In [6]:
df = pd.read_sql(data_query,con=conn)

In [7]:
df.shape

(869260, 15)

In [8]:
df['Key'] = df['Item'] +"-"+ df['Material Type'] +"-"+ df['Div'] +"-"+ df['Cust Facing Loc'] +"-"+ df['Sales Org']+"-"+df['Lag']


In [9]:
df['Date'] = pd.to_datetime(df['Date'])

In [10]:
df = df.convert_dtypes()

In [11]:
df.dropna(inplace=True)

In [12]:
df.drop_duplicates(subset=['Key','Div'],inplace=True)

In [15]:
df.shape

(109986, 16)

In [16]:
df.groupby(['Key'])['Actual','Stat','Consensus','ABS_STAT','ABS_CONS'].sum()